In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


#Importing Libraries

In [2]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import cv2
import numpy as np
from glob import glob


In [1]:
ROOT_PATH=r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images"

In [2]:
pwd

'C:\\Users\\Gen. Shakefire\\Desktop\\Rice Disease Detection Using CNN\\Rice Disease Detection Using CNN'

In [5]:
import os 
os.chdir(ROOT_PATH)
os.getcwd()

'C:\\Users\\Gen. Shakefire\\Desktop\\Rice Leaf Disease Images\\Rice Leaf Disease Images'

In [6]:
train_path=r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\train"
test_path=r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\train"
val_path=r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\validation"

In [7]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)
        img_arr= target_size=(224,224),

        #img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)

In [8]:
x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)
        img_arr= target_size=(224,224),

       # img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

In [9]:
x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr= target_size=(224,224),

        #img_arr=cv2.resize(img_arr,(224,224))

        x_val.append(img_arr)

In [10]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

#Resizing all images

In [11]:
IMAGE_SIZE=[224,224]

#Importing VGG16 library and add preprocesing layer in front of the VGG16.
#Here we will se ImageNet weights

In [12]:
vgg16=VGG16(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

include_top= False means that we will cut off the last dense layer of pre-trained model of VGG16 as it was trained for 1000 output categories of imagenet whereas we only have 3 categories. We also cut off the 1st layer because the input image and the size can be of my choice.

In [13]:
for layers in vgg16.layers:
  print(layers)

<InputLayer name=input_layer, built=True>
<Conv2D name=block1_conv1, built=True>
<Conv2D name=block1_conv2, built=True>
<MaxPooling2D name=block1_pool, built=True>
<Conv2D name=block2_conv1, built=True>
<Conv2D name=block2_conv2, built=True>
<MaxPooling2D name=block2_pool, built=True>
<Conv2D name=block3_conv1, built=True>
<Conv2D name=block3_conv2, built=True>
<Conv2D name=block3_conv3, built=True>
<MaxPooling2D name=block3_pool, built=True>
<Conv2D name=block4_conv1, built=True>
<Conv2D name=block4_conv2, built=True>
<Conv2D name=block4_conv3, built=True>
<MaxPooling2D name=block4_pool, built=True>
<Conv2D name=block5_conv1, built=True>
<Conv2D name=block5_conv2, built=True>
<Conv2D name=block5_conv3, built=True>
<MaxPooling2D name=block5_pool, built=True>


###Setting layer.trainable to False moves all the layer's weights from trainable to non-trainable. This is called "freezing" the layer: the state of a frozen layer won't be updated during training (either when training with fit() or when training with any custom loop that relies on trainable_weights to apply gradient updates).

In [14]:
for layer in vgg16.layers:
  layer.trainable=False

In [15]:
for layer in vgg16.layers:
  print(layer.name,layer.trainable)

input_layer False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False


#Vgg16 Model Summary

In [16]:
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

#Building the Final Model 


In [29]:
model = Sequential()

model.add(vgg16)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))


#View the structure of final model

In [30]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 4)                   │           1,028 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,138,500 (80.64 MB)

 Trainable params: 6,423,812 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

#Specifying the Cost and Optimization Methods

In [31]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#Using the Image Data Generator to import images from dataset

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen= ImageDataGenerator(rescale=1./255,
                                  shear_range =0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen =  ImageDataGenerator(rescale = 1./255)

In [33]:
pwd

'C:\\Users\\Gen. Shakefire\\Desktop\\Rice Leaf Disease Images\\Rice Leaf Disease Images'

In [34]:
training_set=train_datagen.flow_from_directory(r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\train",
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')


Found 2019 images belonging to 4 classes.


In [35]:
test_set=test_datagen.flow_from_directory(r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\train",
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                            )

Found 2019 images belonging to 4 classes.


In [36]:
val_set=val_datagen.flow_from_directory(r'C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\validation',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical'
                                            )

Found 1095 images belonging to 4 classes.


In [37]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [38]:
training_set.class_indices
train_y.shape,test_y.shape,val_y.shape

((2019,), (2019,), (1095,))

#Early stopping to avoid overfitting of model

In [39]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)


#Model Fitting

In [ ]:
r= model.fit(
    training_set,
    validation_data=val_set,
    epochs=20,
    verbose = 1,
    callbacks=[early_stop] 
)   

Epoch 1/20


C:\Users\Gen. Shakefire\anaconda3\envs\Landmine2024\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 2879s 45s/step - accuracy: 0.5753 - loss: 2.0043 - val_accuracy: 0.9142 - val_loss: 0.2498
Epoch 2/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 2842s 45s/step - accuracy: 0.9135 - loss: 0.2561 - val_accuracy: 0.9689 - val_loss: 0.1239
Epoch 3/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 2851s 45s/step - accuracy: 0.9549 - loss: 0.1409 - val_accuracy: 0.9744 - val_loss: 0.0844
Epoch 4/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 2837s 45s/step - accuracy: 0.9756 - loss: 0.0910 - val_accuracy: 0.9635 - val_loss: 0.0905
Epoch 5/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 2845s 45s/step - accuracy: 0.9697 - loss: 0.0860 - val_accuracy: 0.9872 - val_loss: 0.0403
Epoch 6/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 2850s 45s/step - accuracy: 0.9869 - loss: 0.0497 - val_accuracy: 0.9945 - val_loss: 0.0273
Epoch 7/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 2903s 46s/step - accuracy: 0.9927 - loss: 0.0317 - val_accuracy: 0.9534 - val_loss: 0.1302
Epoch 8/20
64/64 ━━━━━━━━━━━━━━━━━━━━ 3233s 51s/step - accuracy: 0.9725 - loss: 0.0841 - val_accuracy: 0.987

#Plotting the Losses

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(r.history['loss'], label='Train Loss')
plt.plot(r.history['val_loss'], label='Val Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

#Plotting Accuracy 


In [ ]:
plt.plot(r.history['accuracy'], label='Train Acc')
plt.plot(r.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

#Saving the File as .h5 Type


In [ ]:
from tensorflow.keras.models import load_model 
model.save('model_vgg16.h5')

#Predictions

In [ ]:
y_pred=model.predict(test_set)


In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
from tensorflow.keras.preprocessing import image


In [ ]:
model=load_model('model_vgg16.h5')
img_path=r"C:\Users\Gen. Shakefire\Desktop\Rice Leaf Disease Images\Rice Leaf Disease Images\train\Blast\BLAST1_011.jpg"
img=image.load_img(img_path,target_size=(224,224))


In [ ]:
x=image.img_to_array(img)

In [ ]:
Z=plt.imread(img_path)

In [ ]:
plt.imshow(Z)

In [ ]:
x.shape


In [ ]:
x=x/255

In [ ]:
from keras.applications.vgg16 import preprocess_input 

In [ ]:
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)

In [ ]:
result=np.argmax(model.predict(img_data),axis=1)

In [ ]:
result

In [ ]:
# Predict on a new image
model=load_model('model_vgg16.h5')
img_path =r"C:\Users\Gen. Shakefire\Desktop\New folder\test\Brownspot\brownspot_rotated_007.jpg"
img=image.load_img(img_path,target_size=(224,224))
img = np.expand_dims(img, axis=0)
img = img / 255.0
prediction = model.predict(img)
predicted_class = np.argmax(prediction, axis=1)
class_labels = list(training_set.class_indices.keys())
predicted_label = class_labels[predicted_class[0]]

In [ ]:
Z=plt.imread(img_path)
plt.title(f"Predicted: {predicted_label}")
plt.imshow(Z)

Losses and Accuracy 


In [ ]:
 from sklearn.metrics import accuracy_score
 from sklearn.metrics import f1_score
 from sklearn.metrics import roc_auc_score


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
f1_score(test_y,y_pred)

In [ ]:
accuracy_score(test_y, y_pred)

In [ ]:
roc_auc_score(test_y,y_pred)

In [ ]:
print(classification_report(y_pred,test_y))

In [ ]:
print(confusion_matrix(y_pred,test_y))